In [1]:
'''
%pip install kagglehub

import kagglehub

#Download latest version
path = kagglehub.dataset_download("yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018")

print("Path to dataset files:", path)

%pip install pandas
%pip install numpy
'''

'\n%pip install kagglehub\n\nimport kagglehub\n\n#Download latest version\npath = kagglehub.dataset_download("yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018")\n\nprint("Path to dataset files:", path)\n\n%pip install pandas\n%pip install numpy\n'

# Data preperation

**In this file we take a look at the dataset focusing on formatting the data for the rest of the work**

**The dataset is located here https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018/data?select=2013.csv**

In [2]:
import pandas as pd
import numpy as np

df=pd.read_csv('2013.csv')

Let's start working with the 2013's data. Let's check at first which features it has by printing the one of the rows and then checking the dataset's dimensions.

In [3]:
print(f"Dimensions of the dataset are: {np.shape(df)}")
print(df.dtypes) # Let's check what the types of each column are and whether it is correct or not

Dimensions of the dataset are: (6369482, 28)
FL_DATE                 object
OP_CARRIER              object
OP_CARRIER_FL_NUM        int64
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY              float64
TAXI_OUT               float64
WHEELS_OFF             float64
WHEELS_ON              float64
TAXI_IN                float64
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
CANCELLATION_CODE       object
DIVERTED               float64
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed: 27            float64
dtype: object


## Formatting

Our dataset has 6369482 rows and 28 columns.

As we can see from above, most numeric features are in float64 format, we ought to standardize the format of all of the numeric features to int64 instead. 

There is an unnecessary column 'Unnamed: 27' representing nothing and consisting only of NaN values. We will remove the column.


In [4]:
def float_to_int(df):
    for column in df.columns:
        if df[column].dtype == 'float64':
            # Convert float to int by rounding
            df[column] = df[column].round().astype('Int64')
    return df

df = float_to_int(df)

df = df.drop(columns=['Unnamed: 27'], errors='ignore') # Removing the useless column

In addition, there are many times in the military time format : HHMM, we will convert them to a new, 'HH:MM' format.

In [5]:
def convert_to_time_format(value):
    if pd.isna(value):
        return pd.NaT
    value = int(value)  # In case it's stored as float
    hours = value // 100
    minutes = value % 100
    return f"{hours:02}:{minutes:02}"

time_columns = ['CRS_DEP_TIME', 'DEP_TIME', 'WHEELS_OFF', 'WHEELS_ON', 'CRS_ARR_TIME', 'ARR_TIME']
# Selecting the columns that should be transformed

for col in time_columns:
    df[col] = df[col].apply(convert_to_time_format) # Applying changes

There are also 2 attributes, which are functionally flags, however are still represented as int64, this is unpreferable in ML applications, we will fix this :

In [6]:
# Convert CANCELLED and DIVERTED to boolean
boolean_columns = ['CANCELLED', 'DIVERTED']
for col in boolean_columns:
    df[col] = df[col].astype(bool)

**Now we check the results of our formatting changes :**

In [7]:
print(df.dtypes)

FL_DATE                object
OP_CARRIER             object
OP_CARRIER_FL_NUM       int64
ORIGIN                 object
DEST                   object
CRS_DEP_TIME           object
DEP_TIME               object
DEP_DELAY               Int64
TAXI_OUT                Int64
WHEELS_OFF             object
WHEELS_ON              object
TAXI_IN                 Int64
CRS_ARR_TIME           object
ARR_TIME               object
ARR_DELAY               Int64
CANCELLED                bool
CANCELLATION_CODE      object
DIVERTED                 bool
CRS_ELAPSED_TIME        Int64
ACTUAL_ELAPSED_TIME     Int64
AIR_TIME                Int64
DISTANCE                Int64
CARRIER_DELAY           Int64
WEATHER_DELAY           Int64
NAS_DELAY               Int64
SECURITY_DELAY          Int64
LATE_AIRCRAFT_DELAY     Int64
dtype: object


Usually it's considered bad etiquette to maintain composite attributes in databases, for this reason, let's split the FL_DATE attribute into it's smaller components:

In [ ]:
# Convert FL_DATE to datetime if not already
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

# Extract month and day
df['MONTH'] = df['FL_DATE'].dt.month
df['DAY'] = df['FL_DATE'].dt.day
#df['YEAR'] = df['FL_DATE'].dt.year
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek  # 0 = Monday, 6 = Sunday

# Optional: Drop original FL_DATE column
df.drop(columns=['FL_DATE'], inplace=True)

# Verify the new columns
print("New columns added:")
print(df[['MONTH', 'DAY', 'YEAR', 'DAY_OF_WEEK']].head())

# Quick statistical overview
print("\nMonth distribution:")
print(df['MONTH'].value_counts(normalize=True).sort_index() * 100)

print("\nDay of week distribution:")
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_dist = df['DAY_OF_WEEK'].value_counts(normalize=True).sort_index() * 100
day_dist.index = day_names
print(day_dist)

New columns added:
   MONTH  DAY  YEAR  DAY_OF_WEEK
0      1    1  2013            1
1      1    1  2013            1
2      1    1  2013            1
3      1    1  2013            1
4      1    1  2013            1

Month distribution:
MONTH
1     7.999379
2     7.374948
3     8.671223
4     8.421297
5     8.613605
6     8.668539
7     8.974403
8     8.837783
9     8.019585
10    8.404828
11    7.901679
12    8.112732
Name: proportion, dtype: float64

Day of week distribution:
Monday       14.901353
Tuesday      14.600230
Wednesday    14.620341
Thursday     14.833530
Friday       14.946788
Saturday     12.050038
Sunday       14.047720
Name: proportion, dtype: float64


## Checking for discrepencies

Let's start checking for discrepencies, we will start by checking the number of missing (null) values in columns.

In [9]:
df.isnull().sum()

OP_CARRIER                   0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_TIME                 91681
DEP_DELAY                91681
TAXI_OUT                 94903
WHEELS_OFF               94903
WHEELS_ON                98275
TAXI_IN                  98275
CRS_ARR_TIME                 0
ARR_TIME                 98275
ARR_DELAY               110172
CANCELLED                    0
CANCELLATION_CODE      6273470
DIVERTED                     0
CRS_ELAPSED_TIME             5
ACTUAL_ELAPSED_TIME     110172
AIR_TIME                110172
DISTANCE                     0
CARRIER_DELAY          5100205
WEATHER_DELAY          5100205
NAS_DELAY              5100205
SECURITY_DELAY         5100205
LATE_AIRCRAFT_DELAY    5100205
MONTH                        0
DAY                          0
YEAR                         0
DAY_OF_WEEK                  0
dtype: int64

It's alarming that amount of missing values in WHEELS_OFF and WHEELS_ON isn't the same. Let's investigate.

In [10]:
# First filter for WHEELS_OFF not missing and WHEELS_ON missing
filtered_df = df[
    (df['WHEELS_OFF'].notna()) & 
    (df['WHEELS_ON'].isna())
]
print(f"Discrepency is present in : {np.shape(filtered_df)}")
# Additional filter for non-cancelled flights
filtered_df = filtered_df[
    (filtered_df['CANCELLED'] == False)
]
print(f"Discrepency after cancellations is present : {np.shape(filtered_df)}")

filtered_df = filtered_df[
    (filtered_df['DIVERTED'] == False)
]
print(f"Discrepency after diversions is present : {np.shape(filtered_df)}")

# Set display option to show all columns
pd.set_option('display.max_columns', None)

Discrepency is present in : (3372, 30)
Discrepency after cancellations is present : (2263, 30)
Discrepency after diversions is present : (0, 30)


The issue is that there is no data about the flight arriving. It is as if the plane has been in the air for years. However, after applying some filters, we discover, that this only happens when the flight was cancelled or diverted, making the discrepency make sense: the plane took off, but didn't land in the destination airport.

In [11]:
df[['DEP_TIME', 'WHEELS_OFF']].isnull().sum()

DEP_TIME      91681
WHEELS_OFF    94903
dtype: int64

*The number of missing values is not the same for 'DEP_TIME' and 'WHEELS_OFF'. Let's investigate:*

In [12]:
# First filter
filtered_df = df[df['DEP_TIME'].notna() & df['WHEELS_OFF'].isna()]
print(np.shape(filtered_df))

# Corrected second filter
filtered_df2 = filtered_df[(filtered_df['CANCELLED'] == False)]
print(np.shape(filtered_df2))

pd.set_option('display.max_columns', None)

(3222, 30)
(0, 30)


It seems like there are cases where airplane has departed but has not touched off the ground. This would mean that airport has been driving on land for years which makes no sense.

However, we can see once again, that all of these flights, were cancelled flights. Making the discrepency somewhat logical: the plane started to move, but was unable to take off for whatever reason.

### Dealing with the Cancelled and Diverted flights

From these discrepencies, we can conclude, that it is more practical to consider these special cases: cancelled and diverted flights, seperately. After all, we cannot apply uniform analysis to normal flights and these exceptions.

In [13]:
df_cancel = df[
    (df['CANCELLED'] == True) | 
    (df['DIVERTED'] == True)
]

print(np.shape(df_cancel))

(110172, 30)


However, this means, that some attributes become useless: the flags and the CANCELLATION_CODE. Just in case, let's first drop the cancelled and diverted flights from the main dataframe and then check if there are any not NaN values in the CANCELLATON_CODE column left

In [14]:
# First, drop cancelled and diverted flights
df_without_cancelled_diverted = df[
    (df['CANCELLED'] == False) & 
    (df['DIVERTED'] == False)
]

# Check for non-NaN values in CANCELLATION_CODE
non_nan_cancellation_codes = df_without_cancelled_diverted[
    df_without_cancelled_diverted['CANCELLATION_CODE'].notna()
]

# Print results
print("Total rows in original dataframe:", len(df))
print("Rows after removing cancelled/diverted flights:", len(df_without_cancelled_diverted))
print("Rows with non-NaN cancellation codes:", len(non_nan_cancellation_codes))

# If you want to see the unique non-NaN cancellation codes
print("\nUnique non-NaN Cancellation Codes:")
print(non_nan_cancellation_codes['CANCELLATION_CODE'].unique())

Total rows in original dataframe: 6369482
Rows after removing cancelled/diverted flights: 6259310
Rows with non-NaN cancellation codes: 0

Unique non-NaN Cancellation Codes:
[]


We can now safely drop all three of these columns : 

In [15]:
# Drop CANCELLED, DIVERTED, and CANCELLATION_CODE columns
df_cleaned = df_without_cancelled_diverted.drop(
    columns=['CANCELLED', 'DIVERTED', 'CANCELLATION_CODE']
)

# Verify the drop
print("Columns before dropping:", list(df_without_cancelled_diverted.columns))
print("Columns after dropping:", list(df_cleaned.columns))

# Optional: Verify the number of rows remains the same
print("Number of rows before dropping columns:", len(df_without_cancelled_diverted))
print("Number of rows after dropping columns:", len(df_cleaned))

Columns before dropping: ['OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'MONTH', 'DAY', 'YEAR', 'DAY_OF_WEEK']
Columns after dropping: ['OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'MONTH', 'DAY', 'YEAR', 'DAY_OF_WEEK']
Number of rows before dropping columns: 6259310
Number of rows after dropping columns: 6259310


Let's check if any discrepancies remain:

In [16]:
df_cleaned.isnull().sum()

OP_CARRIER                   0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_TIME                     0
DEP_DELAY                    0
TAXI_OUT                     0
WHEELS_OFF                   0
WHEELS_ON                    0
TAXI_IN                      0
CRS_ARR_TIME                 0
ARR_TIME                     0
ARR_DELAY                    0
CRS_ELAPSED_TIME             0
ACTUAL_ELAPSED_TIME          0
AIR_TIME                     0
DISTANCE                     0
CARRIER_DELAY          4990033
WEATHER_DELAY          4990033
NAS_DELAY              4990033
SECURITY_DELAY         4990033
LATE_AIRCRAFT_DELAY    4990033
MONTH                        0
DAY                          0
YEAR                         0
DAY_OF_WEEK                  0
dtype: int64

Seemingly the normal flights list is now fixed

### **Missing values in the canceled and diverted flights list**

There are 5 cases in which 'CRS_ELAPSED_TIME' has a missing value. However, it is possible to calculate it by using columns 'CRS_ARR_TIME' and 'CRS_DEP_TIME' which have no missing values.

In [17]:
missing_crs_elapsed_time = df_cancel[df_cancel['CRS_ELAPSED_TIME'].isna()]
print(missing_crs_elapsed_time[['ORIGIN','DEST','CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'CANCELLED', 'DIVERTED']])

#DENver = UTC(-7) 6 a.m.
#BNA Nashville, MDW Chicago = UTC(-6) 7 a.m.
#SNA   UTC(-8) 5 a.m.

        ORIGIN DEST CRS_DEP_TIME CRS_ARR_TIME  CRS_ELAPSED_TIME  CANCELLED  \
1060779    DEN  MDW        08:30        11:46              <NA>      False   
1060780    MDW  DEN        12:25        14:02              <NA>       True   
5389021    SNA  DEN        15:00        18:09              <NA>       True   
5768225    DEN  BNA        10:22        13:48              <NA>      False   
5768226    BNA  DEN        14:28        16:20              <NA>       True   

         DIVERTED  
1060779      True  
1060780     False  
5389021     False  
5768225      True  
5768226     False  


As we see, it is possible to calculate it but we would also need to consider timezone difference so let's check the whole rows.

There are five flights which lack the planned elapsed time, to calculate this, we need to consider the departure and arrival airports and their timezones, for example : Chicago and Nashville are in the same timezone and Denver is 1 hour behind them. This means that the first flight's 'CRS_ELAPSED_TIME' should be 97 + 60 = 157. In a similar manner, we calculate for the other four flights.

In [18]:
df_cancel.loc[df_cancel.index == 1060779, 'CRS_ELAPSED_TIME'] = 136  # DEN to  MDW
df_cancel.loc[df_cancel.index == 1060780, 'CRS_ELAPSED_TIME'] = 157  # MDW to DEN
df_cancel.loc[df_cancel.index == 5768226, 'CRS_ELAPSED_TIME'] = 172  # BNA to DEN
df_cancel.loc[df_cancel.index == 5389021, 'CRS_ELAPSED_TIME'] = 129  # SNA to DEN
df_cancel.loc[df_cancel.index == 5768225, 'CRS_ELAPSED_TIME'] = 146  # DEN to BNA

print(df_cancel.loc[[1060779,1060780, 5768226,5389021,5768225], ['CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME']]) # Checking if the missing values have been filled

        CRS_DEP_TIME CRS_ARR_TIME  CRS_ELAPSED_TIME
1060779        08:30        11:46               136
1060780        12:25        14:02               157
5768226        14:28        16:20               172
5389021        15:00        18:09               129
5768225        10:22        13:48               146


There are still couple problems within our data. At first, amount of missing values in 'ARR_DELAY' is not equal to amount of missing values in 'ARR_TIME'. Let's investigate.

In [19]:
filtered_df = df_cancel[df_cancel['ARR_TIME'].notna() & df_cancel['ARR_DELAY'].isna()] # Filtering rows where ARR_TIME is not missing, and ARR_DELAY is missing

pd.set_option('display.max_columns', None)

print(np.shape(df_cancel))
print(np.shape(filtered_df))

(110172, 30)
(11897, 30)


We can fix those missing values by calculating them. We need to do that for 'ARR_DELAY', 'ACTUAL_ELAPSED_TIME' and 'AIR_TIME'.

After reviewing the format from the dataframe: we can calculate 'ARR_DELAY' by subtracting 'ARR_TIME' from 'CRS_ARR_TIME' and convert the result into minutes. We can calculate 'ACTUAL_ELAPSED_TIME' by adding 'ARR_DELAY' and 'CRS_ELAPSED_TIME'.  We can calculate 'AIR_TIME' by subtracting 'TAXI_IN' and 'TAXI_OUT' from 'ACTUAL_ELAPSED_TIME'. Let's fix this.

In [20]:
filtered_df2 = df_cleaned[df_cleaned['ARR_TIME'].notna() & df_cleaned['ARR_DELAY'].isna()]
print(np.shape(filtered_df2))

(0, 27)


^ There are no normal flights with this discrepency

In [21]:
filtered_df2 = df_cancel[df_cancel['ARR_TIME'].notna() & df_cancel['ARR_DELAY'].isna()]
print(np.shape(filtered_df2))

(11897, 30)


In [22]:
def time_to_minutes(time_str):
    # Split the time string into hours and minutes
    hours, minutes = map(int, time_str.split(':'))
    # Return the total minutes after midnight
    return hours * 60 + minutes

# Creating a copy of filtered_df2 to avoid SettingWithCopyWarning
filtered_df2_copy = filtered_df2.copy()

# Applying the function to 'ARR_TIME' and 'CRS_ARR_TIME' on the copied DataFrame
filtered_df2_copy['ARR_TIME_minutes'] = filtered_df2_copy['ARR_TIME'].apply(time_to_minutes)
filtered_df2_copy['CRS_ARR_TIME_minutes'] = filtered_df2_copy['CRS_ARR_TIME'].apply(time_to_minutes)

# Calculating 'ARR_DELAY' as the difference between 'ARR_TIME' and 'CRS_ARR_TIME'
filtered_df2_copy['ARR_DELAY'] = filtered_df2_copy['ARR_TIME_minutes'] - filtered_df2_copy['CRS_ARR_TIME_minutes']

# Calculating 'ACTUAL_ELAPSED_TIME' by adding 'ARR_DELAY' to 'CRS_ELAPSED_TIME'
filtered_df2_copy['ACTUAL_ELAPSED_TIME'] = filtered_df2_copy['ARR_DELAY'] + filtered_df2_copy['CRS_ELAPSED_TIME']

# Calculating 'AIR_TIME' by subtracting 'TAXI_IN' and 'TAXI_OUT' from 'ACTUAL_ELAPSED_TIME'
filtered_df2_copy['AIR_TIME'] = filtered_df2_copy['ACTUAL_ELAPSED_TIME'] - filtered_df2_copy['TAXI_IN'] - filtered_df2_copy['TAXI_OUT']

# Dropping the temporary columns 'ARR_TIME_minutes' and 'CRS_ARR_TIME_minutes'
filtered_df2_copy.drop(['ARR_TIME_minutes', 'CRS_ARR_TIME_minutes'], axis=1, inplace=True)


print(filtered_df2_copy[['ARR_TIME', 'CRS_ARR_TIME', 'ARR_DELAY', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME']].head(15))
print(filtered_df2_copy.head(15))

     ARR_TIME CRS_ARR_TIME  ARR_DELAY  ACTUAL_ELAPSED_TIME  AIR_TIME
231     10:55        10:10         45                  365       355
846     14:48        13:10         98                  253       229
3149    20:02        16:47        195                  363       328
3384    23:58        22:07        111                  328       305
3614    15:42        12:36        186                  289       268
3739    20:58        14:19        399                  585       548
3845    03:12        22:12      -1140                -1068     -1082
4014    19:19        13:26        353                  433       409
4394    18:44        15:35        189                  276       252
4457    15:20        13:17        123                  237       201
5571    19:02        18:15         47                  187       164
5881    12:54        09:15        219                  399       379
6546    10:15        08:20        115                  225       207
6647    19:34        18:05        

As visible from row with index 3845, this solution does not account for the cases in which plane arrived after midnight but was scheduled to arrive before midnight. After having a look on the data, it can be concluded that investigating those anomalies is tricky. One way is to determine from whole data how long the longest flights in the dataset took. Let's take a look at the most extreme 'CRS_ELAPSED_TIME' and 'ACTUAL_ELAPSED_TIME' values.

In [25]:
# Sorting the dataframe by 'CRS_ELAPSED_TIME' in descending order
max_elapsed_time_row = df_cancel.sort_values(by='CRS_ELAPSED_TIME', ascending=False).head(20)

# Display the row with the maximum elapsed time
print(max_elapsed_time_row[[ 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'CRS_ELAPSED_TIME', 'ARR_TIME', 'CRS_ARR_TIME']])

        OP_CARRIER  OP_CARRIER_FL_NUM  CRS_ELAPSED_TIME ARR_TIME CRS_ARR_TIME
664866          UA                 15               660      NaT        19:41
1427297         UA                145               645      NaT        17:35
1798081         UA                 15               645    22:44        18:15
1070558         UA                145               645      NaT        18:35
4736902         UA                 15               644    19:23        18:13
3700524         UA                 15               636      NaT        18:01
3277255         UA                 15               636    21:49        18:01
3199142         UA                 15               636    21:21        18:01
3700543         UA                145               627      NaT        17:27
2566242         DL                837               590    18:48        14:40
4736901         UA                 14               580    14:23        11:55
3700523         UA                 14               578      NaT

By estimating that flights that are going to be seen as an anomaly won't take over 12 hours (720 minutes) it is possible to classify a case as an anomaly (the flight was supposed to land before midnight but landed after midnight) if 'CRS_ARR_TIME' is from 12:00 to 23:59 and if 'ARR_TIME' is from 00:00 to 11:59. If both of those conditions are true then we should deal by calculating 'ARR_DELAY' in a different way.

In [26]:
filtered_df2_copy['ARR_TIME_minutes'] = filtered_df2_copy['ARR_TIME'].apply(time_to_minutes)
filtered_df2_copy['CRS_ARR_TIME_minutes'] = filtered_df2_copy['CRS_ARR_TIME'].apply(time_to_minutes)

# Identifying rows with anomaly conditions
anomalies = (filtered_df2_copy['CRS_ARR_TIME_minutes'] >= 720) & (filtered_df2_copy['ARR_TIME_minutes'] < 720)

# Adding 1440 minutes to 'ARR_TIME_minutes' for anomalies (arrived after midnight but was not supposed to)
filtered_df2_copy.loc[anomalies, 'ARR_TIME_minutes'] += 1440

# Recalculating 'ARR_DELAY' as the difference between 'ARR_TIME' and 'CRS_ARR_TIME'
filtered_df2_copy['ARR_DELAY'] = filtered_df2_copy['ARR_TIME_minutes'] - filtered_df2_copy['CRS_ARR_TIME_minutes']

# Recalculating 'ACTUAL_ELAPSED_TIME' by adding 'ARR_DELAY' to 'CRS_ELAPSED_TIME'
filtered_df2_copy['ACTUAL_ELAPSED_TIME'] = filtered_df2_copy['ARR_DELAY'] + filtered_df2_copy['CRS_ELAPSED_TIME']

# Recalculating 'AIR_TIME' by subtracting 'TAXI_IN' and 'TAXI_OUT' from 'ACTUAL_ELAPSED_TIME'
filtered_df2_copy['AIR_TIME'] = filtered_df2_copy['ACTUAL_ELAPSED_TIME'] - filtered_df2_copy['TAXI_IN'] - filtered_df2_copy['TAXI_OUT']

# Dropping the temporary columns 'ARR_TIME_minutes' and 'CRS_ARR_TIME_minutes'
filtered_df2_copy.drop(['ARR_TIME_minutes', 'CRS_ARR_TIME_minutes'], axis=1, inplace=True)

# Print the updated dataframe with the calculated columns
print(filtered_df2_copy[['ARR_TIME', 'CRS_ARR_TIME', 'ARR_DELAY', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME']].head(15))
print(filtered_df2_copy.head(15))

     ARR_TIME CRS_ARR_TIME  ARR_DELAY  ACTUAL_ELAPSED_TIME  AIR_TIME
231     10:55        10:10         45                  365       355
846     14:48        13:10         98                  253       229
3149    20:02        16:47        195                  363       328
3384    23:58        22:07        111                  328       305
3614    15:42        12:36        186                  289       268
3739    20:58        14:19        399                  585       548
3845    03:12        22:12        300                  372       358
4014    19:19        13:26        353                  433       409
4394    18:44        15:35        189                  276       252
4457    15:20        13:17        123                  237       201
5571    19:02        18:15         47                  187       164
5881    12:54        09:15        219                  399       379
6546    10:15        08:20        115                  225       207
6647    19:34        18:05        

Now we will apply the changes to the dataset we want to continue working with (df_cleaned2).

In [27]:
# Using .loc[] to ensure we are modifying the original DataFrame correctly
df_cancel.loc[df_cancel['ARR_DELAY'].isna(), 'ARR_DELAY'] = filtered_df2_copy['ARR_DELAY']
df_cancel.loc[df_cancel['ACTUAL_ELAPSED_TIME'].isna(), 'ACTUAL_ELAPSED_TIME'] = filtered_df2_copy['ACTUAL_ELAPSED_TIME']
df_cancel.loc[df_cancel['AIR_TIME'].isna(), 'AIR_TIME'] = filtered_df2_copy['AIR_TIME']

# Checking if it worked
print(df_cancel[['ARR_DELAY', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME']].head(15))
print(df_cancel.loc[3845])


      ARR_DELAY  ACTUAL_ELAPSED_TIME  AIR_TIME
231          45                  365       355
846          98                  253       229
2878       <NA>                 <NA>      <NA>
2973       <NA>                 <NA>      <NA>
3130       <NA>                 <NA>      <NA>
3131       <NA>                 <NA>      <NA>
3132       <NA>                 <NA>      <NA>
3149        195                  363       328
3245       <NA>                 <NA>      <NA>
3310       <NA>                 <NA>      <NA>
3322       <NA>                 <NA>      <NA>
3325       <NA>                 <NA>      <NA>
3367       <NA>                 <NA>      <NA>
3384        111                  328       305
3422       <NA>                 <NA>      <NA>
OP_CARRIER                EV
OP_CARRIER_FL_NUM       4703
ORIGIN                   IAH
DEST                     BRO
CRS_DEP_TIME           21:00
DEP_TIME               23:30
DEP_DELAY                150
TAXI_OUT                   9
WHEELS_OFF      

Now there are only few things left to check - whether amount of missing values in 'CANCELLATION_CODE' makes sense and whether the amounts of missing values in 'CARRIER_DELAY',
'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY' and 'LATE_AIRCRAFT_DELAY' make sense.

In [28]:
df_cleaned.isnull().sum()

OP_CARRIER                   0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_TIME                     0
DEP_DELAY                    0
TAXI_OUT                     0
WHEELS_OFF                   0
WHEELS_ON                    0
TAXI_IN                      0
CRS_ARR_TIME                 0
ARR_TIME                     0
ARR_DELAY                    0
CRS_ELAPSED_TIME             0
ACTUAL_ELAPSED_TIME          0
AIR_TIME                     0
DISTANCE                     0
CARRIER_DELAY          4990033
WEATHER_DELAY          4990033
NAS_DELAY              4990033
SECURITY_DELAY         4990033
LATE_AIRCRAFT_DELAY    4990033
MONTH                        0
DAY                          0
YEAR                         0
DAY_OF_WEEK                  0
dtype: int64

In [29]:
# Counting how many times a flight was not cancelled
cancelled_count = df_cancel[df_cancel['CANCELLED'] == 1].shape[0]
diverted_count = df_cancel[df_cancel['DIVERTED'] == 1].shape[0]

print(f'Number of Cancelled flights: {cancelled_count}')
print(f'Number of Diverted flights: {diverted_count}')

Number of Cancelled flights: 96012
Number of Diverted flights: 14160


## Adding attributes

### Rush Hour

In [30]:
'''
import matplotlib.pyplot as plt
import seaborn as sns

# Ensuring 'CRS_DEP_TIME' is in the correct string format and extracting the hour
df_cleaned['DEP_Hour'] = df_cleaned['CRS_DEP_TIME'].str.split(':').str[0].astype(int)

# Plotting the distribution of DEP_Hour
plt.figure(figsize=(12, 6))
ax = sns.histplot(
    df_cleaned['DEP_Hour'], 
    bins=range(25), 
    kde=False, 
    color='blue', 
    shrink=0.9  # Adjusts the bar width
)

# Adding text annotations inside each bar, vertically, only for counts > 100000
for p in ax.patches:
    count = int(p.get_height())  # Getting the height of each bar
    if count > 100000:  # Only annotating bars with counts over 100000
        ax.annotate(
            f'{count}',
            (p.get_x() + p.get_width() / 2., count / 2),  # Centering inside the bar
            ha='center', va='center', fontsize=10, color='white', weight='bold',
            rotation=90  # Makes the text vertical
        )


plt.title('Distribution of Scheduled Departure Hours (CRS_DEP_TIME)', fontsize=14)
plt.xlabel('Hour of Day (24-Hour Clock)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(range(0, 25))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
'''

"\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Ensuring 'CRS_DEP_TIME' is in the correct string format and extracting the hour\ndf_cleaned['DEP_Hour'] = df_cleaned['CRS_DEP_TIME'].str.split(':').str[0].astype(int)\n\n# Plotting the distribution of DEP_Hour\nplt.figure(figsize=(12, 6))\nax = sns.histplot(\n    df_cleaned['DEP_Hour'], \n    bins=range(25), \n    kde=False, \n    color='blue', \n    shrink=0.9  # Adjusts the bar width\n)\n\n# Adding text annotations inside each bar, vertically, only for counts > 100000\nfor p in ax.patches:\n    count = int(p.get_height())  # Getting the height of each bar\n    if count > 100000:  # Only annotating bars with counts over 100000\n        ax.annotate(\n            f'{count}',\n            (p.get_x() + p.get_width() / 2., count / 2),  # Centering inside the bar\n            ha='center', va='center', fontsize=10, color='white', weight='bold',\n            rotation=90  # Makes the text vertical\n        )\n\n\nplt.title('Distrib

From this plot, it is possible to see that the distribution of planned departure time is very similar from 6:00 to 19:59. Timespan 17:00 - 17:59 had the most planned departures (446554) and it was closely followed by timespan 8:00 - 8:59 (442666 planned departures). Third most popular timespan for planned departures was 7:00 - 7:59 with 432303 planned departures. Let's classify only timespans 17:00 - 17:59 and 8:00 - 8:59 as rush hours for departures and later use this information to see if departure time at rush hour caused bigger delay.

Let's create a binary feature 'DEP_Rush_Hour' that has value 1 when scheduled departure time was in timespans 8:00 - 8:59 or 17:00 - 17:59. We can use previously defined feature 'DEP_Hour' to define this feature.

In [31]:
Rush_Hour = [8, 17]
#Rush_Hour.__contains__(x)
#df_cleaned['DEP_Rush_Hour'] = df_cleaned['DEP_Hour'].apply(lambda x: 1 if x. == 8 or x == 17 else 0)
#df_cleaned2[['DEP_Hour', 'DEP_Rush_Hour']].head(20) # Checking if it worked

Now, let's repeat the process for arrivals but now focus on actual arrival times, not scheduled arrival times. Reasoning for focusing on actual arrival times is that results from further analysis will be more realistic.

In [32]:
'''
# Creating the 'ARR_Hour' feature, extract hour only for non-NaT values
df_cleaned2['ARR_Hour'] = df_cleaned2['ARR_TIME'].apply(lambda x: int(x.split(':')[0]) if pd.notna(x) else pd.NaT)



# Plotting the distribution of DEP_Hour
plt.figure(figsize=(12, 6))
ax = sns.histplot(
    df_cleaned2['ARR_Hour'], 
    bins=range(25), 
    kde=False, 
    color='blue', 
    shrink=0.9  # Adjusts the bar width
)

# Adding text annotations inside each bar, vertically, only for counts > 100000
for p in ax.patches:
    count = int(p.get_height())  # Getting the height of each bar
    if count > 100000:  # Only annotating bars with counts over 100000
        ax.annotate(
            f'{count}',
            (p.get_x() + p.get_width() / 2., count / 2),  # Centering inside the bar
            ha='center', va='center', fontsize=10, color='white', weight='bold',
            rotation=90  # Makes the text vertical
        )


plt.title('Distribution of Actual Arrival Hours (ARR_TIME)', fontsize=14)
plt.xlabel('Hour of Day (24-Hour Clock)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(range(0, 25))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
'''

"\n# Creating the 'ARR_Hour' feature, extract hour only for non-NaT values\ndf_cleaned2['ARR_Hour'] = df_cleaned2['ARR_TIME'].apply(lambda x: int(x.split(':')[0]) if pd.notna(x) else pd.NaT)\n\n\n\n# Plotting the distribution of DEP_Hour\nplt.figure(figsize=(12, 6))\nax = sns.histplot(\n    df_cleaned2['ARR_Hour'], \n    bins=range(25), \n    kde=False, \n    color='blue', \n    shrink=0.9  # Adjusts the bar width\n)\n\n# Adding text annotations inside each bar, vertically, only for counts > 100000\nfor p in ax.patches:\n    count = int(p.get_height())  # Getting the height of each bar\n    if count > 100000:  # Only annotating bars with counts over 100000\n        ax.annotate(\n            f'{count}',\n            (p.get_x() + p.get_width() / 2., count / 2),  # Centering inside the bar\n            ha='center', va='center', fontsize=10, color='white', weight='bold',\n            rotation=90  # Makes the text vertical\n        )\n\n\nplt.title('Distribution of Actual Arrival Hours (ARR

In [33]:
# Selecting only relevant numerical features for summary statistics
numerical_features = ['DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'ARR_DELAY', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 
    'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']

# Generating descriptive statistics for numerical features
numerical_stats = df_cleaned[numerical_features].describe()

# Displaying the statistics
numerical_stats


,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,6259310.0,6259310.0,6259310.0,6259310.0,6259310.0,6259310.0,6259310.0,6259310.0,1269277.0,1269277.0,1269277.0,1269277.0,1269277.0
mean,9.684819,15.592381,6.772477,6.010872,132.994205,129.320258,106.9554,764.57842,16.652706,2.343187,13.729758,0.081661,23.868734
std,35.567039,9.033458,4.710094,38.052002,72.169934,71.66801,69.769858,585.970127,43.581631,17.025016,27.65401,2.627572,41.037532
min,-171.0,1.0,1.0,-153.0,20.0,11.0,5.0,31.0,0.0,0.0,0.0,0.0,0.0
25%,-5.0,10.0,4.0,-12.0,81.0,78.0,57.0,341.0,0.0,0.0,0.0,0.0,0.0
50%,-1.0,13.0,6.0,-3.0,114.0,110.0,87.0,596.0,1.0,0.0,3.0,0.0,7.0
75%,9.0,18.0,8.0,10.0,163.0,159.0,136.0,991.0,16.0,0.0,18.0,0.0,31.0
max,1975.0,237.0,346.0,1983.0,700.0,738.0,695.0,4983.0,1975.0,1591.0,1287.0,573.0,1182.0


## Wrapping up

**The data formatting is done, we will save the two dataframes into files and utilize them in the next notebooks on EDA and ML**

In [34]:
df_cleaned.to_csv('df_cleaned.csv', index=False)
df_cancel.to_csv('df_cancel.csv', index=False)